In [1]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.7/358.7 kB 9.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 34.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 107.5 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.4/284.4 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 15.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.8/899.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
  

In [5]:
from unsloth import FastLanguageModel

/tmp/ipykernel_47/2814113929.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth Zoo will now patch everything to make training faster!


In [6]:
import json
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer

In [7]:
# --- Configuration ---
DATA_PATH = "/kaggle/input/interiit-25-observe-ai-transcripts/transcripts.json"      # Path to your JSON file
NUM_SAMPLES = 35                    # Number of diverse transcripts to send to LLM
# MODEL_ID = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit" # Local 4-bit quantized model
MODEL_ID = "unsloth/Qwen3-14B-unsloth-bnb-4bit"
# MODEL_ID = "Qwen/Qwen2.5-7B-Instruct-AWQ"       # Alternative
EMBEDDING_BATCH_SIZE = 512          # Batch size for generating embeddings (adjust based on VRAM)


In [8]:
LLM_MODEL, tokenizer = FastLanguageModel.from_pretrained(
    model_name=MODEL_ID,
    max_seq_length=4096*4,
    dtype=None,
    load_in_4bit=True,
    device_map = "balanced"
)
FastLanguageModel.for_inference(LLM_MODEL)  # ✅ Enable faster inference

==((====))==  Unsloth 2025.11.4: Fast Qwen3 patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.59G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 5120, padding_idx=151654)
    (layers): ModuleList(
      (0-5): 6 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=17408, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=17408, bias=False)
          (down_proj): Linear4bit(in_features=17408, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
   

In [24]:
# --- Helper: Convert JSON Transcript to String ---
def transcript_to_string(record):
    """
    Converts the structured conversation list into a readable string.
    Includes metadata like 'reason_for_call' to help the embedding model.
    """
    # 1. Start with metadata context (Very helpful for clustering)
    text_representation = f"Reason for call: {record.get('reason_for_call', 'N/A')}\n"
    
    # 2. Append the dialogue
    for turn in record['conversation']:
        text_representation += f"{turn['speaker']}: {turn['text']}\n"
        
    return text_representation

# --- Step 1: Smart Diversity Sampling ---
def get_diverse_samples(json_data, n_samples=50):
    print(f"--- 1. Embeddings & Clustering (Total Records: {len(json_data)}) ---")
    
    # 1. Prepare texts
    print("Preparing text representations...")
    all_texts = [transcript_to_string(record) for record in json_data]
    
    # 2. Embed using a lightweight local model with GPU acceleration
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Embedding transcripts using all-MiniLM-L6-v2 on {device} with batch size {EMBEDDING_BATCH_SIZE}...")
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2', device=device)
    embeddings = embedder.encode(
        all_texts, 
        batch_size=EMBEDDING_BATCH_SIZE, 
        show_progress_bar=True,
        convert_to_numpy=True
    )
    
    # 3. Cluster to find diversity
    print(f"Clustering into {n_samples} clusters...")
    kmeans = KMeans(n_clusters=n_samples, random_state=42, n_init=10)
    kmeans.fit(embeddings)
    
    # 4. Find the transcript closest to the center of each cluster
    closest, _ = pairwise_distances_argmin_min(kmeans.cluster_centers_, embeddings)
    
    # selected_records = [json_data[i] for i in closest]
    selected_texts = [all_texts[i] for i in closest]
    
    print(f"Selected {len(selected_texts)} diverse transcripts.")
    return selected_texts

# --- Step 2: LLM Taxonomy Generation ---
def generate_taxonomy_with_llm(sample_texts):
    print("\n--- 2. Loading Quantized LLM ---")
    
    # 1. Load 4-bit Quantized Model
    # device_map="auto" automatically uses GPU if available
    # model, tokenizer = FastLanguageModel.from_pretrained(
    #     model_name=MODEL_ID,
    #     # max_seq_length=4096*2,
    #     dtype=None,
    #     load_in_4bit=True,
    # )
    # FastLanguageModel.for_inference(LLM_MODEL)  # ✅ Enable faster inference
    
    print("LLM Loaded. Generating Taxonomy...")

    # 2. Prepare the prompt (Chunking if necessary, but 50 short transcripts usually fit 8k context)
    # If 50 is too large for context, we take the first 15-20 most distinct ones.
    # Let's use the first 15 for the prompt to be safe on context limits.
    prompt_samples = sample_texts[:25]
    
    formatted_samples = ""
    for i, text in enumerate(prompt_samples):
        formatted_samples += f"\n--- TRANSCRIPT {i+1} ---\n{text[:1500]}..." # Truncate slightly if super long

    system_prompt = """You are an expert Data Taxonomist building an 'Intent Graph' for a retrieval system.
    
    Your task is to analyze customer service transcripts and generate a **Hierarchical Intent Taxonomy**.
    
    ### DEFINITIONS:
    1. **Primary Intent**: The broad category of the conversation (e.g., Billing, Scheduling).
    2. **Secondary Intent**: The specific granular action taken in a single turn.
       - **User Intents**: What the customer is trying to do (e.g., "Reject Offer", "Complain about Delay").
       - **Agent Intents**: What the agent is trying to do (e.g., "Propose Alternative", "Verify Identity").
    
    ### FEW-SHOT EXAMPLES (Follow this granularity):
    
    **Example 1: Scheduling Domain**
    Output JSON:
    {
      "Service Scheduling": {
         "user": ["Request Appointment", "Reject Proposed Time", "Confirm Time"],
         "agent": ["Propose Time Slot", "Offer Alternative Location", "Confirm Booking"]
      }
    }
    
    **Example 2: Billing Domain**
    Output JSON:
    {
      "Billing & Refunds": {
         "user": ["Dispute Charge", "Request Refund Status", "Threaten Churn"],
         "agent": ["Explain Charge", "Process Refund", "Deny Request Policy"]
      }
    }

    ### INSTRUCTIONS:
    - Analyze the provided transcripts to discover the Domain-Specific intents.
    - **CRITICAL:** You MUST generate both 'user' and 'agent' lists for every Primary Intent.
    - Output ONLY valid JSON.
    """
    
    user_msg = f"""Here are diverse transcripts from the dataset. Analyze them and produce the JSON taxonomy.
    
    {formatted_samples}
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_msg},
    ]

    # 3. Generate
    # Explicitly move inputs to CUDA to ensure they match the model device
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True, enable_thinking=True
    )
    inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to(LLM_MODEL.device)
 
    generated_ids = LLM_MODEL.generate(
        **inputs,  # ✅ Unpack inputs dict
        max_new_tokens=2048,
        temperature=0.1,
        do_sample=True
    )
    output_ids = generated_ids[0][len(inputs.input_ids[0]):].tolist() 
    # return "response" to directly get model outputs string
    # response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
    return output_ids

In [10]:
# --- Main Execution Flow ---
# 1. Load Data
try:
    with open(DATA_PATH, 'r') as f:
        full_data = json.load(f)
        # Handle case where json is a list of records directly
        # or if it's wrapped in a key like {"transcripts": [...]}
        if isinstance(full_data, dict) and "transcripts" in full_data:
            full_data = full_data["transcripts"]
except FileNotFoundError:
    print(f"Error: {DATA_PATH} not found. Please ensure the file exists.")
    exit()

In [11]:
# 2. Get Diverse Samples (The "Smart 100")
diverse_transcript_texts = get_diverse_samples(full_data, n_samples=NUM_SAMPLES)

--- 1. Embeddings & Clustering (Total Records: 19621) ---
Preparing text representations...
Embedding transcripts using all-MiniLM-L6-v2 on cuda with batch size 512...


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/39 [00:00<?, ?it/s]

Clustering into 35 clusters...
Selected 35 diverse transcripts.


In [ ]:
# prompt_samples = diverse_transcript_texts[:50] 

# formatted_samples = ""
# for i, text in enumerate(prompt_samples):
#     formatted_samples += f"\n--- TRANSCRIPT {i+1} ---\n{text[:1500]}..." # Truncate slightly if super long

# system_prompt = """You are an expert Data Taxonomist.
# Your task is to analyze these customer service transcripts and generate a **Domain-Agnostic Intent Taxonomy**.

# Identify:
# 1. **Primary Intents**: The high-level category of the conversation (e.g., Billing, Technical Support, Sales).
# 2. **Secondary Intents**: Specific actions taken by the User or Agent (e.g., "Ask for Discount", "Reject Offer", "Clarify Policy").

# **CRITICAL OUTPUT RULES:**
# - Output ONLY valid JSON.
# - Do not write intro/outro text.
# - Ensure 'Secondary Intents' cover both Customer and Agent actions found in the transcripts.

# **JSON Structure:**
# {
#   "primary_intents": [
#     "Category A",
#     "Category B"
#   ],
#   "secondary_intents": {
#     "Category A": ["Action 1", "Action 2"],
#     "Category B": ["Action 3", "Action 4"]
#   }
# }
# """

# user_msg = f"""Here are diverse transcripts from the dataset. Analyze them and produce the JSON taxonomy.

# {formatted_samples}
# """

# messages = [
#     {"role": "system", "content": system_prompt},
#     {"role": "user", "content": user_msg},
# ]

# # 3. Generate
# # Explicitly move inputs to CUDA to ensure they match the model device
# prompt = tokenizer.apply_chat_template(
#     messages, tokenize=True, add_generation_prompt=True
# )

# print(f'total tokens: {len(prompt)}')

In [25]:
# 3. Generate Taxonomy
output_ids = generate_taxonomy_with_llm(diverse_transcript_texts)


--- 2. Loading Quantized LLM ---
LLM Loaded. Generating Taxonomy...


In [26]:
# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
taxonomy_json_str = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

In [27]:
# 4. Save Output
print("\n--- Generated Taxonomy ---")
print(taxonomy_json_str)


--- Generated Taxonomy ---
{
  "Billing & Refunds": {
    "user": ["Dispute Charge", "Request Refund Status", "Threaten Churn", "Inquire About Billing Discrepancies"],
    "agent": ["Explain Charge", "Process Refund", "Deny Request Policy", "Investigate Billing Issue"]
  },
  "Insurance Renewal": {
    "user": ["Inquire About Renewal Options", "Express Concern About Costs", "Request Policy Clarification"],
    "agent": ["Explain Policy Options", "Discuss Incentives", "Clarify Coverage Limits"]
  },
  "Order Delivery": {
    "user": ["Check Delivery Status", "Complain About Delay", "Request Carrier Info"],
    "agent": ["Provide Delivery Update", "Offer Carrier Info", "Apologize for Delays"]
  },
  "Flight Rebooking": {
    "user": ["Request Rebooking Options", "Inquire About Compensation", "Express Urgency"],
    "agent": ["Provide Rebooking Options", "Offer Compensation", "Explain Delay Policies"]
  },
  "Service Outage": {
    "user": ["Report Outage", "Ask About Resolution Time", "

In [28]:
# Clean code block markers if LLM adds them
clean_json = taxonomy_json_str.replace("```json", "").replace("```", "").strip()

with open("taxonomy.json", "w") as f:
    f.write(clean_json)
print("\n[Saved] Taxonomy saved to taxonomy.json")


[Saved] Taxonomy saved to taxonomy.json
